# The Robot Javelin Puzzle: A Game Theory Simulation

This notebook explores the solution to the Robot Javelin puzzle from Jane Street. We'll use Python and `numpy` to simulate the game, find the Nash Equilibrium of the base game, and then analyze the impact of an information leak.

## Part 1: The Base Game and Nash Equilibrium

The base game is as follows:

1.  Two players (robots) each get a score `X` drawn uniformly from `[0, 1]`.
2.  Independently and without knowing the other's score, each player decides whether to **keep** their score or **reroll`.
3.  A reroll gives a new score, also drawn uniformly from `[0, 1]`, which must be kept.
4.  The player with the higher final score wins.

We're looking for a symmetric Nash Equilibrium strategy. This is a strategy that, if both players adopt it, neither player has an incentive to deviate. The strategy is defined by a cutoff value `t`: if the initial score `X` is greater than or equal to `t`, the player keeps it; otherwise, they reroll.

### Finding the Equilibrium Cutoff `t*`

At the equilibrium cutoff `t*`, a player must be indifferent between keeping the score `t*` and rerolling. We can find this `t*` by simulating the game for various values of `t` and finding which `t` makes the expected payoff of keeping versus rerolling the same.

The code below simulates the game and searches for this equilibrium.

In [ ]:
import numpy as np

def simulate_game(t, n_sims=100000, eps=1e-4):
    """Simulates the base game to find the equilibrium cutoff t."""
    # Player 1's and 2's first draws
    x1 = np.random.rand(n_sims)
    x2 = np.random.rand(n_sims)

    # Player 1's decision
    reroll1 = x1 < t
    final1 = np.where(reroll1, np.random.rand(n_sims), x1)

    # Player 2's decision
    reroll2 = x2 < t
    final2 = np.where(reroll2, np.random.rand(n_sims), x2)

    # Payoffs
    p1_wins = (final1 > final2) + 0.5 * (final1 == final2)

    # Indifference condition at the cutoff t
    # We look at a small band around t for Player 1's first draw
    indifference_band = (x1 >= t - eps) & (x1 < t + eps)
    
    if np.sum(indifference_band) == 0:
        # No data in the band, cannot estimate difference
        return np.mean(p1_wins), np.nan

    # Scenario A: Player 1 keeps the score in the band
    # Player 2 plays as usual
    p2_reroll_if_kept = x2 < t
    p2_final_if_kept = np.where(p2_reroll_if_kept, np.random.rand(np.sum(indifference_band)), x2[indifference_band])
    win_prob_if_keep = np.mean((x1[indifference_band] > p2_final_if_kept) + 0.5 * (x1[indifference_band] == p2_final_if_kept))

    # Scenario B: Player 1 rerolls
    p1_reroll_val = np.random.rand(np.sum(indifference_band))
    p2_reroll_if_reroll = x2 < t
    p2_final_if_reroll = np.where(p2_reroll_if_reroll, np.random.rand(np.sum(indifference_band)), x2[indifference_band])
    win_prob_if_reroll = np.mean((p1_reroll_val > p2_final_if_reroll) + 0.5 * (p1_reroll_val == p2_final_if_reroll))

    diff = win_prob_if_keep - win_prob_if_reroll
    return np.mean(p1_wins), diff

def search_equilibrium_t(T_grid, n_sims_per_t):
    """Searches for the equilibrium t over a grid."""
    diffs = []
    for t in T_grid:
        _, diff = simulate_game(t, n_sims_per_t)
        diffs.append(diff)
    
    diffs = np.array(diffs)
    best_t_idx = np.nanargmin(np.abs(diffs))
    best_t = T_grid[best_t_idx]
    diff_at_best_t = diffs[best_t_idx]
    return best_t, diff_at_best_t

# Search for the equilibrium t*
T_grid = np.linspace(0.6, 0.7, 51)
t_star, diff_t_star = search_equilibrium_t(T_grid, n_sims_per_t=100000)

# The analytical solution is t = (1 + sqrt(3)) / 4, which is approx 0.683
t_star_analytical = (1 + np.sqrt(3)) / 4

print(f"Approximate equilibrium t*: {t_star:.4f}")
print(f"Win prob difference at t*: {diff_t_star:.6f}")
print(f"Analytical equilibrium t*: {t_star_analytical:.4f}")

Our simulation finds an equilibrium cutoff `t*` very close to the analytical solution of `(1 + sqrt(3)) / 4`, which is approximately 0.683. This gives us confidence in our simulation approach.

## Part 2: The Information Leak

Now, Spears Robot gets an advantage. Before deciding to reroll, Spears learns whether Java-lin's first throw was above or below a threshold `d` that Spears chooses.

Spears' strategy now depends on this information. We model this with two cutoffs:
- `t_low`: Spears' cutoff if Java-lin's first throw is `< d`.
- `t_high`: Spears' cutoff if Java-lin's first throw is `>= d`.

Java-lin, unaware of the leak, continues to use the `t*` we found earlier.

### Finding Spears' Optimal Strategy

We need to find the parameters `(d, t_low, t_high)` that Spears would choose to maximize their win rate (and thus minimize Java-lin's). We'll search over a grid of possible values for these parameters.

In [ ]:
def spears_vs_java(t_java, d, t_low, t_high, n_sims=100000):
    """Simulates the game with Spears' information advantage."""
    # Java-lin's first throw
    x_j = np.random.rand(n_sims)

    # Spears' first throw
    x_s = np.random.rand(n_sims)

    # Java-lin's decision (unaware of the leak)
    reroll_j = x_j < t_java
    final_j = np.where(reroll_j, np.random.rand(n_sims), x_j)

    # The information leak bit for Spears
    b = x_j >= d

    # Spears' decision, based on the bit
    spears_cutoff = np.where(b, t_high, t_low)
    reroll_s = x_s < spears_cutoff
    final_s = np.where(reroll_s, np.random.rand(n_sims), x_s)

    # Payoffs
    java_lin_wins = (final_j > final_s) + 0.5 * (final_j == final_s)
    return np.mean(java_lin_wins)

# Grid search for Spears' best strategy
d_grid = np.linspace(0.1, 0.9, 9)       # Coarse grid for speed
t_low_grid = np.linspace(0.1, 0.9, 9)
t_high_grid = np.linspace(0.1, 0.9, 9)

best_spears_params = {}
min_java_win_rate = 1.0

t_java = t_star_analytical # Assume Java-lin uses the analytical t*

for d in d_grid:
    for t_low in t_low_grid:
        for t_high in t_high_grid:
            win_rate = spears_vs_java(t_java, d, t_low, t_high, n_sins=10000) # Lower sims for grid search
            if win_rate < min_java_win_rate:
                min_java_win_rate = win_rate
                best_spears_params = {'d': d, 't_low': t_low, 't_high': t_high}

print(f"Spears' best parameters: {best_spears_params}")
print(f"Java-lin's win rate against this strategy: {min_java_win_rate:.6f}")

## Part 3: Java-lin's Counter-Strategy

Now, we know that Spears is using this optimal strategy. Java-lin can react by adjusting its own cutoff, `t_java`. We want to find the `t_java` that maximizes Java-lin's win rate against Spears' fixed optimal strategy.

In [ ]:
# Now, we find Java-lin's best response
t_java_grid = np.linspace(0.1, 0.9, 81)
best_java_win_rate = 0.0
t_java_adjusted = 0.0

# Spears uses the best strategy we found
d_opt = best_spears_params['d']
t_low_opt = best_spears_params['t_low']
t_high_opt = best_spears_params['t_high']

for t_java in t_java_grid:
    win_rate = spears_vs_java(t_java, d_opt, t_low_opt, t_high_opt, n_sims=100000)
    if win_rate > best_java_win_rate:
        best_java_win_rate = win_rate
        t_java_adjusted = t_java

print(f"Java-lin's adjusted cutoff: {t_java_adjusted:.4f}")
print(f"Java-lin's final win probability: {best_java_win_rate:.10f}")

## Conclusion

By simulating the game under these asymmetric conditions, we can estimate Java-lin's best possible win rate. The simulation suggests that by adjusting its cutoff, Java-lin can slightly improve its chances, even with Spears' informational advantage.